A German NGO has documented all attacks on pride parades in 2024: https://www.amadeu-antonio-stiftung.de/csds-schuetzen/ I am scraping their page to build a database I can use for analysis and mapping. 

In [1]:
from bs4 import BeautifulSoup
import requests

response = requests.get("https://www.amadeu-antonio-stiftung.de/csds-schuetzen/")
doc = BeautifulSoup(response.text)

I can't find a way to get all the information (place, description, state and date) at once so I am doing this in two steps and then merging the dataframes.

In [2]:
items = doc.find_all(class_="story-content")
for item in items:
    print("------")
    print(item.find('h3').text)
    print(item.find('p').text)

------
Euskirchen
Eine Gruppe Jugendlicher beleidigt und bespuckt zwei Personen am Bahnhof, die für den CSD angereist sind. Bereits im Vorfeld gibt es Anfeindungen und Drohungen auf Facebook.
------
Flensburg
Wenige Tage vor der Veranstaltung stehlen Unbekannte mehrere Regenbogen-Banner und verbrennen sie. Bereits im Vorfeld sammeln sich in den Sozialen Medien unter den Ankündigungsposts des CSDs Hasskommentare.
------
Hannover
Es gab mehrere gewalttätige Angriffe. Am Rande des Demozuges bewerfen Jugendliche eine Person mit Steinen. Am Tag darauf wird eine Gruppe Frauen von einem 43-Jährigen belästigt, am Abend eine 24-Jährige von einem Unbekannten bedrängt und diskriminiert.
------
Weißwasser
Im Vorfeld werden mehrfach gewaltsam Großplakate zerstört, die auf den zweiten CSD in der Stadt aufmerksam machen sollen.
------
Dresden
Die rechtsextreme Gruppierung „Elblandrevolte“ ruft auf Social Media dazu auf, den CSD in Dresden zu stören. Nachdem die Polizei bei Personenkontrollen Reichsfa

In [3]:
rows = []

for item in items:
    row = {}
    row['place'] = item.find('h3').text
    row['description'] = item.find('p').text

    rows.append(row)
len(rows)

55

In [4]:
import pandas as pd

df1 = pd.json_normalize(rows)
df1.head()

,place,description
0,Euskirchen,Eine Gruppe Jugendlicher beleidigt und bespuck...
1,Flensburg,Wenige Tage vor der Veranstaltung stehlen Unbe...
2,Hannover,Es gab mehrere gewalttätige Angriffe. Am Rande...
3,Weißwasser,Im Vorfeld werden mehrfach gewaltsam Großplaka...
4,Dresden,Die rechtsextreme Gruppierung „Elblandrevolte“...


In [5]:
items = doc.find_all(class_="story-time")
for item in items:
    print("------")
    print(item.text)

------
17.05.2024Nordrhein-Westfalen
------
17.05.2024Schleswig-Holstein
------
18.05.2024Niedersachsen
------
25.5.2024Sachsen
------
01.06.2024Sachsen
------
01.06.2024Nordrhein-Westfalen
------
01.06.2024Schleswig-Holstein
------
08.06.2024Bayern
------
08.06.2024Mecklenburg-Vorpommern
------
08.06.2024Baden-Württemberg
------
08.06.2024Sachsen
------
15.06.2024Sachsen-Anhalt
------
22.06.2024Brandenburg
------
22.06.2024Bayern
------
22.06.2024Thüringen
------
29.06.2024Baden-Württemberg
------
06.07.2024Brandenburg
------
06.07.2024Bayern
------
13.07.2024Thüringen
------
13.07.2024Baden-Württemberg
------
13.07.2024Baden-Württemberg
------
20.07.2024Thüringen
------
21.07.2024Nordrhein-Westfalen
------
27.07.2024Berlin
------
27.07.2024Nordrhein-Westfalen
------
03.08.2024Nordrhein-Westfalen
------
10.08.2024Sachsen
------
10.08.2024Niedersachsen
------
10.08.2024Nordrhein-Westfalen
------
17.08.2024Rheinland-Pfalz
------
17.08.2024Sachsen
------
17.08.2024Nordrhein-Westfalen
---

In [6]:
rows = []

for item in items:
    row = {}
    row['date-and-state'] = item.find('p').text

    rows.append(row)
len(rows)

55

In [7]:
df2 = pd.json_normalize(rows)
df2.head()

,date-and-state
0,17.05.2024Nordrhein-Westfalen
1,17.05.2024Schleswig-Holstein
2,18.05.2024Niedersachsen
3,25.5.2024Sachsen
4,01.06.2024Sachsen


Now I have to extract the state name and the date to make two separate columns.

In [8]:
df2['date'] = df2['date-and-state'].str.extract(r'(\d{1,2}\.\d{1,2}\.\d{4})')

In [9]:
df2

,date-and-state,date
0,17.05.2024Nordrhein-Westfalen,17.05.2024
1,17.05.2024Schleswig-Holstein,17.05.2024
2,18.05.2024Niedersachsen,18.05.2024
3,25.5.2024Sachsen,25.5.2024
4,01.06.2024Sachsen,01.06.2024
5,01.06.2024Nordrhein-Westfalen,01.06.2024
6,01.06.2024Schleswig-Holstein,01.06.2024
7,08.06.2024Bayern,08.06.2024
8,08.06.2024Mecklenburg-Vorpommern,08.06.2024
9,08.06.2024Baden-Württemberg,08.06.2024


In [11]:
df2['state'] = df2['date-and-state'].str.extract(r'\d{1,2}\.\d{1,2}\.\d{4}(.+)', expand=False)

In [12]:
df2

,date-and-state,date,state
0,17.05.2024Nordrhein-Westfalen,17.05.2024,Nordrhein-Westfalen
1,17.05.2024Schleswig-Holstein,17.05.2024,Schleswig-Holstein
2,18.05.2024Niedersachsen,18.05.2024,Niedersachsen
3,25.5.2024Sachsen,25.5.2024,Sachsen
4,01.06.2024Sachsen,01.06.2024,Sachsen
5,01.06.2024Nordrhein-Westfalen,01.06.2024,Nordrhein-Westfalen
6,01.06.2024Schleswig-Holstein,01.06.2024,Schleswig-Holstein
7,08.06.2024Bayern,08.06.2024,Bayern
8,08.06.2024Mecklenburg-Vorpommern,08.06.2024,Mecklenburg-Vorpommern
9,08.06.2024Baden-Württemberg,08.06.2024,Baden-Württemberg


Checking if any were missed

In [14]:
df2.isna()

,date-and-state,date,state
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
7,False,False,False
8,False,False,False
9,False,False,False


In [15]:
df2 = df2.drop(columns=['date-and-state'])

In [16]:
df2.head()

,date,state
0,17.05.2024,Nordrhein-Westfalen
1,17.05.2024,Schleswig-Holstein
2,18.05.2024,Niedersachsen
3,25.5.2024,Sachsen
4,01.06.2024,Sachsen


The dates are not coherent so I'm converting the column to datetime.

In [17]:
df2['date'] = pd.to_datetime(df2['date'], dayfirst=True, errors='coerce')

In [18]:
df2.head()

,date,state
0,2024-05-17,Nordrhein-Westfalen
1,2024-05-17,Schleswig-Holstein
2,2024-05-18,Niedersachsen
3,2024-05-25,Sachsen
4,2024-06-01,Sachsen


In [19]:
joined_df = df1.join(df2)

In [20]:
joined_df.head()

,place,description,date,state
0,Euskirchen,Eine Gruppe Jugendlicher beleidigt und bespuck...,2024-05-17,Nordrhein-Westfalen
1,Flensburg,Wenige Tage vor der Veranstaltung stehlen Unbe...,2024-05-17,Schleswig-Holstein
2,Hannover,Es gab mehrere gewalttätige Angriffe. Am Rande...,2024-05-18,Niedersachsen
3,Weißwasser,Im Vorfeld werden mehrfach gewaltsam Großplaka...,2024-05-25,Sachsen
4,Dresden,Die rechtsextreme Gruppierung „Elblandrevolte“...,2024-06-01,Sachsen


In [21]:
joined_df.to_csv('csd-attacks.csv', index=False)